In [25]:
import pandas as pd

In [39]:
snd = pd.read_csv("sales_and_dates.csv", parse_dates=['Date'])

In [37]:
snd

,Date,SalePrice
0,11/16/2006 0:00,66000.0
1,3/26/2004 0:00,57000.0
2,2/26/2004 0:00,10000.0
3,5/19/2011 0:00,38500.0
4,7/23/2009 0:00,11000.0
...,...,...
412693,3/7/2012 0:00,10000.0
412694,1/28/2012 0:00,10500.0
412695,1/28/2012 0:00,12500.0
412696,3/7/2012 0:00,10000.0
